#### Install required libraries and tools

In [ ]:
!pip install rouge.score nltk py7zr
!pip install accelerate -U
!pip install transformers -U
!pip install datasets
!pip install torch torchvision torchaudio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.0 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=cb5a4423ae2e0c792536fa287c400c18c26833fe5329ffb0df6045ac794859e7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge.score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import transformers
import datasets
from datasets import load_dataset, load_metric, load_from_disk, load_metric, Dataset
import numpy as np
import nltk
from transformers import BartTokenizer, BartForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Let's import our dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews_clean5.csv to reviews_clean5.csv


In [ ]:
df = pd.read_csv('reviews_clean5.csv')
df

,Unnamed: 0,Id,Summary,Text
0,0,1,good quality dog food,i have bought several of the vitality c...
1,1,2,not as advertised,product arrived labeled as jumbo salted ...
2,2,3,delight says it all,this is a confection that has been aroun...
3,3,4,cough medicine,if you are looking for the secret ingre...
4,4,5,great taffy,great taffy at a great price there was a...
...,...,...,...,...
67587,74626,74627,rich but very satisfying,this brand not only lives up to its na...
67588,74627,74628,the best,for years i have been a huge starbucks c...
67589,74628,74629,favorite coffee,this gevalia one of my favorites coffee ...
67590,74629,74630,great coffee,i really like gevalia colombian roast it ...


In [ ]:
df = pd.read_csv('reviews_clean5.csv',nrows=70000)
pd.set_option('display.max_colwidth',None)
df.drop_duplicates(subset=['Text'], inplace=True)
df.dropna(axis=0,inplace=True)
df.replace('',np.nan,inplace=True)

max_text_len=150
max_summary_len=10

cleaned_text =np.array(df['Text'])
cleaned_summary=np.array(df['Summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

df=pd.DataFrame({'text':short_text,'summary':short_summary})

def remove_short(text):
  tokens = text.split()
  longs = []
  for i in tokens:
     if len(i)>1:
        longs.append(i)
  return (" ".join(longs))

clean_summary = []
for t in df['summary']:
   clean_summary.append(remove_short(t))

df['summary'] = clean_summary

df = df[df['summary'].str.len()>=2]

In [ ]:
df['id'] = df.index
df

,text,summary,id
0,i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most,good quality dog food,0
1,product arrived labeled as jumbo salted peanutsthe peanuts were actually small sized unsalted not sure if this was an error or if the vendor intended to represent the product as jumbo,not as advertised,1
2,this is a confection that has been around a few centuries it is a light pillowy citrus gelatin with nuts in this case filberts and it is cut into tiny squares and then liberally coated with powdered sugar and it is a tiny mouthful of heaven not too chewy and very flavorful i highly recommend this yummy treat if you are familiar with the story of cs lewis the lion the witch and the wardrobe this is the treat that seduces edmund into selling out his brother and sisters to the witch,delight says it all,2
3,if you are looking for the secret ingredient in robitussin i believe i have found it i got this in addition to the root beer extract i ordered and made some cherry soda the flavor is very medicinal,cough medicine,3
4,great taffy at a great price there was a wide assortment of yummy taffy delivery was very quick if your a taffy lover this is a deal,great taffy,4
...,...,...,...
58809,this brand not only lives up to its name but it also brews very well there is no bitter after taste and it leaves one feeling very satisfied only after one cup,rich but very satisfying,58809
58810,for years i have been a huge starbucks coffee lover devoted infact but i was out of coffee and the small grocery store i went to did not carry starbucks so i debated what to get and grab this brand i waited a few days to try it i was sceptic but now i am in love with gevalia coffee and will probably buy it from now onit is smooth tasteful and beyond heaven,the best,58810
58811,this gevalia one of my favorites coffee good flavor and aroma and good price hope that it had more aroma but still great,favorite coffee,58811
58812,i really like gevalia colombian roast it has so rich aroma and great taste one of my favorite drink it several times a day,great coffee,58812


In [ ]:
df

,text,summary,id
0,i have bought several of the vitality canned dog food products and have found them all to be of good quality the product looks more like a stew than a processed meat and it smells better my labrador is finicky and she appreciates this product better than most,good quality dog food,0
1,product arrived labeled as jumbo salted peanutsthe peanuts were actually small sized unsalted not sure if this was an error or if the vendor intended to represent the product as jumbo,not as advertised,1
2,if you are looking for the secret ingredient in robitussin i believe i have found it i got this in addition to the root beer extract i ordered and made some cherry soda the flavor is very medicinal,cough medicine,2
3,great taffy at a great price there was a wide assortment of yummy taffy delivery was very quick if your a taffy lover this is a deal,great taffy,3
4,i got a wild hair for taffy and ordered this five pound bag the taffy was all very enjoyable with many flavors watermelon root beer melon peppermint grape etc my only complaint is there was a bit too much red black licorice flavored pieces between me my kids and my husband this lasted only two weeks i would recommend this brand of taffy it was a delightful treat,nice taffy,4
...,...,...,...
701,not hot at all like the other low star reviewer i got suckered by seeing habenero in the name and the 12 oz size some people might like the flavor citrus y and it has some fish sauce in the mix but if you are looking for hot this are not it,not hot,701
702,i have to admit i was a sucker for the large quantity 12 oz when shopping for hot sauces but now seeing the size of the bottle it reminds of wing sauce bottle sizes plastic bottle it does have a convenient squirt top but overall not very hot or tasty and made mostly from jalape ntilde os if i had seen the ingredients list i would not have bought it jalapenoswatervinegarbrown sugarlime juicefish saucecilantrohabanerogarlicspice blendsaltpotassium sorbatexanthan gum,not hot not habanero,702
703,i never in my life tasted such a good babka its crazy good this is the real babka that my gram mother use to make,best babka,703
704,i am so convinced these are human animal crackers they look and taste exactly like human ones and they contain sugar which is not one of the last ingredients in the nutrition facts either so ill just give my dog very small amounts,my dog loves these but,704


In [ ]:
metric = load_metric('rouge')
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])


<ipython-input-7-373b65c73954>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [ ]:
train = Dataset.from_dict(train)
val = Dataset.from_dict(validate)
test = Dataset.from_dict(test)
df = datasets.DatasetDict({'train':train,'validate':val,'test':test})
df

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 5881
    })
})

In [ ]:
max_input = 150
max_target = 10
batch_size = 16

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
# generic tokenizer class

NameError: ignored

In [ ]:
def preprocess_data(data_to_process):
  #get all the dialogues
  #print(data_to_process.columns)
  inputs = [dialogue for dialogue in data_to_process['text']]
  #tokenize the dialogues

  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [ ]:
tokenize_data = df.map(preprocess_data, batched = True,remove_columns=['id','text','summary'])
tokenize_data

Map:   0%|          | 0/47043 [00:00<?, ? examples/s]

NameError: ignored

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

NameError: ignored

In [ ]:
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
#####################
# metrics
# compute rouge for evaluation
#####################

def compute_rouge(pred):
  predictions, labels = pred

  #decode the predictions
  predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
args = Seq2SeqTrainingArguments(
    'conversation-summ', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=1,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True #available only with CUDA
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validate'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge,
   # remove_unused_columns=False
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
conversation = tokenize_data['test']

In [ ]:
model_inputs = tokenizer(conversation,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])
tokenizer.decode(raw_pred[0])

In [ ]:
trainer.save_model('./bartmodelepoch1/')

In [ ]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
input_text = "mary and jane wants to eat cake but they think they are fat so they did not eat the cake"
input_encoding = tokenizer(input_text, return_tensors="pt")


In [ ]:
summary_ids = model.generate(input_encoding["input_ids"], num_beams=4, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary:", summary)


Summary: mary and jane wants to eat cake but they think they are fat so they did not eat the cake. Mary and Jane want to eat a cake but think they're fat so don't eat it. They don't know why they're not eating it so they don't get it.


In [ ]:
input_encoding

{'input_ids': tensor([[    0, 22501,     8,  1236,  1728,  1072,     7,  3529,  8492,    53,
            51,   206,    51,    32,  5886,    98,    51,   222,    45,  3529,
             5,  8492,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews_without_tok.csv to reviews_without_tok.csv


In [ ]:
df = pd.read_csv('reviews_without_tok.csv',nrows=10)
df

,Unnamed: 0,text,summary
0,0,bought several vitality canned dog food produc...,good quality dog food
1,1,product arrived labeled jumbo salted peanutsth...,advertised
2,2,confection around centuries light pillowy citr...,delight says
3,3,looking secret ingredient robitussin believe f...,cough medicine
4,4,great taffy great price wide assortment yummy ...,great taffy
5,5,got wild hair taffy ordered five pound bag taf...,nice taffy
6,6,saltwater taffy great flavors soft chewy candy...,great good expensive brands
7,7,taffy good soft chewy flavors amazing would de...,wonderful tasty taffy
8,8,right mostly sprouting cats eat grass love rot...,yay barley
9,9,healthy dog food good digestion also good smal...,healthy dog food


In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, tokenizer, input_texts, target_texts):
        self.tokenizer = tokenizer
        self.input_texts = input_texts
        self.target_texts = target_texts

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        input_encoding = self.tokenizer(input_text, padding='max_length', max_length=512, return_tensors='pt', truncation=True)
        target_encoding = self.tokenizer(target_text, padding='max_length', max_length=128, return_tensors='pt', truncation=True)

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'decoder_input_ids': target_encoding['input_ids'].squeeze(),
            'decoder_attention_mask': target_encoding['attention_mask'].squeeze()
        }

# Example data
input_texts = df['text']  # List of input texts
target_texts = df['summary']  # List of corresponding target (summary) texts

# Create dataset and data loader
dataset = CustomDataset(tokenizer, input_texts, target_texts)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [ ]:
import torch
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 1

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        decoder_input_ids = batch['decoder_input_ids']
        decoder_attention_mask = batch['decoder_attention_mask']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, labels=decoder_input_ids)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")

# Save the fine-tuned model
output_model_dir = "./fine_tuned_bart_summarizer"
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_bart_summarizer"  # Path to the saved fine-tuned model
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)

# Input text for which you want to generate a summary
input_text = "This is an example of a longer input text that you want to summarize."

# Tokenize and encode the input text using the tokenizer
input_encoding = tokenizer(input_text, return_tensors="pt")

# Generate the summary
summary_ids = model.generate(input_encoding["input_ids"], num_beams=4, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Summary:", summary)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews_clean5.csv to reviews_clean5.csv


In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('reviews_clean5.csv',nrows=1000)

In [ ]:
df.drop_duplicates(subset=['Text'], inplace=True)
df.dropna(axis=0,inplace=True)
df.replace('',np.nan,inplace=True)

In [ ]:
max_text_len=100
max_summary_len=10

cleaned_text =np.array(df['Text'])
cleaned_summary=np.array(df['Summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])

df=pd.DataFrame({'text':short_text,'summary':short_summary})

def remove_short(text):
  tokens = text.split()
  longs = []
  for i in tokens:
     if len(i)>1:
        longs.append(i)
  return (" ".join(longs))

clean_summary = []
for t in df['summary']:
   clean_summary.append(remove_short(t))

df['summary'] = clean_summary

df = df[df['summary'].str.len()>=2]

In [ ]:
df['id'] = df.index
df

,text,summary,id
0,i have bought several of the vitality c...,good quality dog food,0
1,product arrived labeled as jumbo salted ...,not as advertised,1
2,this is a confection that has been aroun...,delight says it all,2
3,if you are looking for the secret ingre...,cough medicine,3
4,great taffy at a great price there was a...,great taffy,4
...,...,...,...
787,not hot at all like the other low star ...,not hot,787
788,i have to admit i was a sucker for the ...,not hot not habanero,788
789,i never in my life tasted such a good ...,best babka,789
790,i am so convinced these are human anima...,my dog loves these but,790


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk, load_metric, Dataset
import numpy as np
import nltk
import datasets
nltk.download('punkt')

In [ ]:
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-xsum'

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:

train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])


AttributeError: ignored

In [ ]:
train = Dataset.from_dict(train)
val = Dataset.from_dict(validate)
test = Dataset.from_dict(test)
df = datasets.DatasetDict({'train':train,'validate':val,'test':test})
df

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['text', 'summary', 'id'],
        num_rows: 5881
    })
})

In [ ]:
max_input = 150
max_target = 10
batch_size = 8

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
# generic tokenizer class

In [ ]:
def preprocess_data(data_to_process):
  #get all the dialogues
  #print(data_to_process.columns)
  inputs = [dialogue for dialogue in data_to_process['text']]
  #tokenize the dialogues

  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)

  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [ ]:
df['summary'].to_string()

In [ ]:
tokenize_data = df.map(preprocess_data, batched = True,remove_columns=['id','text','summary'])
tokenize_data

Map:   0%|          | 0/47043 [00:00<?, ? examples/s]

Map:   0%|          | 0/5880 [00:00<?, ? examples/s]

Map:   0%|          | 0/5881 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5881
    })
})

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5881
    })
})

In [ ]:
#sample the data
# train_sample = tokenize_data['train'].shuffle(seed=123).select(range(1000))
# validation_sample = tokenize_data['validation'].shuffle(seed=123).select(range(500))
# test_sample = tokenize_data['test'].shuffle(seed=123).select(range(200))

train_sample = tokenize_data['train'].shuffle(seed=123)
validation_sample = tokenize_data['validate'].shuffle(seed=123)
test_sample = tokenize_data['test'].shuffle(seed=123)

In [ ]:
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47043
    })
    validate: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5880
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5881
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5880
    })
})

In [ ]:
train_sample['labels']

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [ ]:
!pip install accelerate -U
!pip install transformers -U

In [ ]:
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
#####################
# metrics
# compute rouge for evaluation
#####################

def compute_rouge(pred):
  #print(pred)
  predictions, labels = pred

  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 633
    })
    validate: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 79
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 80
    })
})

In [ ]:
args = Seq2SeqTrainingArguments(
    'conversation-summ', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=1, # virtually increases batchsize during training
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True #available only with CUDA
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validate'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge,
   # remove_unused_columns=False
)

In [ ]:
trainer.train() # 17 aug

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.508800,1.882738,16.291500,5.235000,15.530200,15.542800,12.000000
2,1.215100,1.814240,16.369700,5.409200,15.709200,15.694600,12.006300
3,1.005800,1.835806,15.829700,5.341600,15.242200,15.233800,12.001700


TrainOutput(global_step=17643, training_loss=1.290873594499733, metrics={'train_runtime': 9823.3619, 'train_samples_per_second': 14.367, 'train_steps_per_second': 1.796, 'total_flos': 4.48009724694528e+16, 'train_loss': 1.290873594499733, 'epoch': 3.0})

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,1.910086,14.779200,3.478500,14.543400,14.531600,12.873400
2,1.888300,1.934664,15.963900,2.580600,14.261500,14.360400,12.531600
2,1.888300,2.175654,14.923800,3.042700,14.616800,14.787500,12.962000


TrainOutput(global_step=948, training_loss=1.4143965515909316, metrics={'train_runtime': 10709.8417, 'train_samples_per_second': 0.177, 'train_steps_per_second': 0.089, 'total_flos': 401252961484800.0, 'train_loss': 1.4143965515909316, 'epoch': 3.0})

In [ ]:
trainer.train() # 10sep

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.510500,1.777544,16.053500,5.324900,15.470500,15.470000,12.003600
2,1.214900,1.761812,16.038200,5.365900,15.374500,15.363600,12.001500
3,1.010700,1.865940,16.148200,5.290100,15.526100,15.518600,12.007000


TrainOutput(global_step=17643, training_loss=1.29336784651225, metrics={'train_runtime': 8412.7644, 'train_samples_per_second': 16.776, 'train_steps_per_second': 2.097, 'total_flos': 4.48009724694528e+16, 'train_loss': 1.29336784651225, 'epoch': 3.0})

In [ ]:
model_inputs = tokenizer(conversation,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])


In [ ]:
tokenize_data['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5881
})

In [ ]:
test_sample[:10]

In [ ]:

# predict_results = trainer.predict(test_sample[:10],max_length=150,num_beams=3)
# metrics = predict_results.metrics

# predictions = np.where(predict_results.predictions != -100, predict_results.predictions, tokenizer.pad_token_id)
# predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
# predictions = [pred.strip() for pred in predictions]


for i in range(1050,1060):
   conversation = df['test']['text'][i]
   model_inputs = tokenizer(conversation,  max_length=max_input, padding='max_length', truncation=True)
   raw_pred, _, _ = trainer.predict([model_inputs])
   print(tokenizer.decode(raw_pred[0]))

</s><s><s><s>my dog loves these greenies etc</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>my new favorite soup from ame</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>no brainer pros and cons not</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>easy to make and tastes great etc</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>great coffee great price great gift etc</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>delicious and crunchie vs</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>fig newtons are the best etc</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>very pleased with the quality ofe</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>my dog will not eat these vs</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


</s><s><s><s>delicious and scrumpt delicious</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
df['test']['summary'][1050:1060]

['my dog loves greenies',
 'favorite microwavable soup',
 'great product wish there was more flavor in it',
 'lotta little bowls',
 'emeril coffee',
 'what think about the pirate booty white cheddar',
 'great snack convenience',
 'giant chewy sweet tarts',
 'have seen my dog eat everything but these',
 'yummmmmmy']

In [ ]:
df['test']['text'][1050:1060]

['my  dog shadow loves  greenies  more  than  any  treat he  is  a pomeranian  and  gets  the  teenies and  are  we  in  trouble  if  we  run  out i  just  bought  the  tub  of  96  on  amazon  for  less  than  half  the  money  that  the  small  packages  cost  at  the  store debbie',
 'i  decided  to  roll  the  dice  and  bought  spring  onion  and  split  pea  soup  since  i like  drinking  soup  and  i was  not  disappointed but  a little  scared  because  this  is  my  first  time  ordering  food  type  items  from  amazon my  sister  thought  i was  crazy  because  this  is  something  i normally  do  not  do this  is  by  far  my  favorite  tried  the  pea  soup  and  it  was  a little  bland  for  my  taste  compared  to  this  ambrosia i  will  try  again  tomorrow  with  some  seasoning  but  this  soup  was  perfect it  was  very  light  and  tasted  good  normally  i expected  very  little  when  it  comes  to  cup  of  noodle  soups  but  this  was  the  best  microwavabl

In [ ]:
metrics

{'test_loss': 1.8355858325958252,
 'test_rouge1': 16.0612,
 'test_rouge2': 5.2683,
 'test_rougeL': 15.4995,
 'test_rougeLsum': 15.5014,
 'test_gen_len': 12.0012,
 'test_runtime': 466.2288,
 'test_samples_per_second': 12.614,
 'test_steps_per_second': 1.579}

In [ ]:
tokenizer.decode(raw_pred[0])

'</s><s><s><s>chocolate and mary love it</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'